<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/time_series_modeling_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 9th July, 2021
# Ref: https://github.com/philipperemy/keras-attention-mechanism
# Objective:
#            i) Use a multivariate time-series data to make prediction
#               for future.
#           ii) Using lstm and attention in time-series modeling
#            

For example, given the stock prices data for last 10 days, as:<br> 
> PrevClose,Open,High,Low,Last,VWAP,Volume_Turnover,Close <br>
Make predictions of Close price for the next day. Here, we use all eight features of the past to make predictions for future.

## Install library

In [62]:
# 0.0 Install attention
! pip install attention

## Call libraries

In [63]:
# 1.0 Usual libraries
import numpy as np

# 1.1 tensorflow related libraries
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from attention import Attention

In [64]:
# 1.2 Display multiple command outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Generate some sample data
Data has just one feature time-value

In [66]:
# 2.0 Take past 10 time steps and make prediction for the next
#       time step
num_samples, time_steps, input_dim, output_dim = 100, 10, 8, 1
data_x = np.random.uniform(size=(num_samples, time_steps, input_dim))
data_y = np.random.uniform(size=(num_samples, output_dim))

In [67]:
# 2.1 Observe data
print(data_x.shape)
print(data_x[:2])   # Show 2 sets of 10 time-steps each

(100, 10, 8)
[[[0.17675323 0.93801267 0.02409247 0.6477727  0.65776365 0.59317151
   0.08562314 0.109912  ]
  [0.83054184 0.75506272 0.83984152 0.17851744 0.80153445 0.03904354
   0.67577563 0.62588049]
  [0.51336595 0.21246101 0.46662786 0.32600182 0.05321681 0.09254027
   0.14650615 0.99589022]
  [0.47477344 0.33267873 0.16787462 0.71668207 0.9588422  0.29327231
   0.43083154 0.71412131]
  [0.51889625 0.21943644 0.90111213 0.3590204  0.16148471 0.58737068
   0.18956921 0.46883456]
  [0.88719905 0.72865888 0.01624707 0.83167424 0.34635637 0.78909589
   0.18802614 0.71327504]
  [0.96604545 0.25194566 0.47482444 0.21534709 0.15669736 0.53637902
   0.45263331 0.26555538]
  [0.77219807 0.53777421 0.45891954 0.43563277 0.96274734 0.08044768
   0.00638809 0.14604222]
  [0.24421238 0.07640809 0.27816959 0.17043745 0.45818475 0.59449068
   0.36071397 0.71290895]
  [0.46937886 0.11690715 0.14521054 0.54835216 0.55051635 0.76433001
   0.89642641 0.96522258]]

 [[0.31481929 0.98680302 0.98971504

In [68]:
# 2.2 Show next 4-values to be predicted
print(data_y[:4])   # Show 4 values

[[0.56864943]
 [0.1047481 ]
 [0.66283023]
 [0.18586409]]


### Normalise data

In [69]:
# 3.0 Reshape data
data_x = data_x.reshape(1000,8)

In [70]:
# 3.1 Global mean of each one of all eight columns
mean = data_x.mean(axis = 0)
data_x -= mean

# Std of each one of eight columns
std = data_x.std(axis = 0)
data_x /= std
data_x = data_x.reshape(100,10,8)

### Our model

In [71]:
# 4.0 Define the model.
model_input = Input(shape=(time_steps, input_dim))
x = LSTM(64, return_sequences=True)(model_input)
x = Attention(32)(x)
x = Dense(1)(x)
model = Model(model_input, x)

In [72]:
# 4.1 Compile and print model summary
model.compile(loss='mae', optimizer='adam')
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 10, 8)]      0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 10, 64)       18688       input_4[0][0]                    
__________________________________________________________________________________________________
last_hidden_state (Lambda)      (None, 64)           0           lstm_3[0][0]                     
__________________________________________________________________________________________________
attention_score_vec (Dense)     (None, 10, 64)       4096        lstm_3[0][0]                     
____________________________________________________________________________________________

In [73]:
# 4.2 Start learning
model.fit(data_x, data_y, epochs=10)

Epoch 1/10
4/4 [==============================] - 2s 10ms/step - loss: 0.4917
Epoch 2/10
4/4 [==============================] - 0s 12ms/step - loss: 0.4126
Epoch 3/10
4/4 [==============================] - 0s 9ms/step - loss: 0.3552
Epoch 4/10
4/4 [==============================] - 0s 9ms/step - loss: 0.3024
Epoch 5/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2563
Epoch 6/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2468
Epoch 7/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2453
Epoch 8/10
4/4 [==============================] - 0s 16ms/step - loss: 0.2388
Epoch 9/10
4/4 [==============================] - 0s 10ms/step - loss: 0.2284
Epoch 10/10
4/4 [==============================] - 0s 10ms/step - loss: 0.2227


In [74]:
# 4.3 Make predictions
pred1 = model.predict(data_x)

### Save model for future

In [75]:
# 5.0 Save model for future
model.save('test_model.h5')

In [76]:
# 5.1 Delete earlier model
del model

# 5.2 Load the saved model
model_h5 = load_model('test_model.h5')

In [77]:
# 5.3 Make predictions with the sabed model
pred2 = model_h5.predict(data_x)

In [78]:
# 5.4 Compare saved model predictions 
#      with predictions made before saving

np.testing.assert_almost_equal(pred1, pred2)
print('Success.')

Success.


In [ ]:
########## I am done ###############